In [44]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [45]:
## Load the dataset
data = pd.read_csv('Churn_Modelling.csv')
data.head()

RowNumber  CustomerId   Surname  CreditScore Geography  Gender  Age  \
0          1    15634602  Hargrave          619    France  Female   42   
1          2    15647311      Hill          608     Spain  Female   41   
2          3    15619304      Onio          502    France  Female   42   
3          4    15701354      Boni          699    France  Female   39   
4          5    15737888  Mitchell          850     Spain  Female   43   

   Tenure    Balance  NumOfProducts  HasCrCard  IsActiveMember  \
0       2       0.00              1          1               1   
1       1   83807.86              1          0               1   
2       8  159660.80              3          1               0   
3       1       0.00              2          0               0   
4       2  125510.82              1          1               1   

   EstimatedSalary  Exited  
0        101348.88       1  
1        112542.58       0  
2        113931.57       1  
3         93826.63       0  
4         79084.10       0

In [46]:
## Preprocess the data
## Drop irrelevant columns
data = data.drop(['RowNumber',"CustomerId","Surname"], axis = 1)
data.head()

CreditScore Geography  Gender  Age  Tenure    Balance  NumOfProducts  \
0          619    France  Female   42       2       0.00              1   
1          608     Spain  Female   41       1   83807.86              1   
2          502    France  Female   42       8  159660.80              3   
3          699    France  Female   39       1       0.00              2   
4          850     Spain  Female   43       2  125510.82              1   

   HasCrCard  IsActiveMember  EstimatedSalary  Exited  
0          1               1        101348.88       1  
1          0               1        112542.58       0  
2          1               0        113931.57       1  
3          0               0         93826.63       0  
4          1               1         79084.10       0

In [47]:
## Encode categorical variable
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

In [48]:
## Onehot encode 'Geography'
from sklearn.preprocessing import OneHotEncoder
one_hot_encoder = OneHotEncoder()
geo_encoder = one_hot_encoder.fit_transform(data[["Geography"]])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [49]:
one_hot_encoder.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [50]:
geo_encoded_df = pd.DataFrame(geo_encoder.toarray(), columns = one_hot_encoder.get_feature_names_out(['Geography']))

In [51]:
geo_encoded_df

Geography_France  Geography_Germany  Geography_Spain
0                  1.0                0.0              0.0
1                  0.0                0.0              1.0
2                  1.0                0.0              0.0
3                  1.0                0.0              0.0
4                  0.0                0.0              1.0
...                ...                ...              ...
9995               1.0                0.0              0.0
9996               1.0                0.0              0.0
9997               1.0                0.0              0.0
9998               0.0                1.0              0.0
9999               1.0                0.0              0.0

[10000 rows x 3 columns]

In [52]:
## Combine one hot encoded columns with the original data
data = pd.concat([data.drop('Geography', axis = 1), geo_encoded_df], axis=1)
data.head()

CreditScore  Gender  Age  Tenure    Balance  NumOfProducts  HasCrCard  \
0          619       0   42       2       0.00              1          1   
1          608       0   41       1   83807.86              1          0   
2          502       0   42       8  159660.80              3          1   
3          699       0   39       1       0.00              2          0   
4          850       0   43       2  125510.82              1          1   

   IsActiveMember  EstimatedSalary  Exited  Geography_France  \
0               1        101348.88       1               1.0   
1               1        112542.58       0               0.0   
2               0        113931.57       1               1.0   
3               0         93826.63       0               1.0   
4               1         79084.10       0               0.0   

   Geography_Germany  Geography_Spain  
0                0.0              0.0  
1                0.0              1.0  
2                0.0              0.0  
3                0.0              0.0  
4                0.0              1.0

In [53]:
## Save the encoders and scaler

with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)
    
with open('one_hot_encoder.pkl', 'wb') as file:
    pickle.dump(one_hot_encoder, file)

In [54]:
## Divide the dataset into indepandent and dependent features
X = data.drop('Exited', axis = 1)
y = data['Exited']

In [55]:
## Split the data into training and testing sets

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2, random_state=42)

## Scale theses features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [56]:
X_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [57]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

In [58]:
X_train.shape[1]

12

### ANN Implementation

In [59]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [60]:
## Build ANN Model
model = Sequential([
    Dense(1024, activation = "relu", input_shape = (X_train.shape[1],)),
    Dense(512, activation = "relu"),
    Dense(128, activation = "relu"),
    Dense(64, activation = "relu"),
    Dense(32, activation = "relu"),
    Dense(16, activation = "relu"),
    Dense(1, activation = "sigmoid")
])

In [61]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_25 (Dense)            (None, 1024)              13312     
                                                                 
 dense_26 (Dense)            (None, 512)               524800    
                                                                 
 dense_27 (Dense)            (None, 128)               65664     
                                                                 
 dense_28 (Dense)            (None, 64)                8256      
                                                                 
 dense_29 (Dense)            (None, 32)                2080      
                                                                 
 dense_30 (Dense)            (None, 16)                528       
                                                                 
 dense_31 (Dense)            (None, 1)                

In [62]:
model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001),
    loss = "binary_crossentropy",
    metrics = ['accuracy']
)

In [63]:
## Tensorboard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir = log_dir, histogram_freq=1)

In [64]:
early_stopping_callbacks = EarlyStopping(
    monitor = 'val_loss',
    patience = 20,
    restore_best_weights = True
)

In [65]:
history = model.fit(
    X_train,
    y_train,
    validation_data = (X_test,y_test),
    epochs = 100,
    callbacks = [tensorflow_callback, early_stopping_callbacks]
)

Epoch 1/100
250/250 [==============================] - 4s 10ms/step - loss: 0.4105 - accuracy: 0.8303 - val_loss: 0.3451 - val_accuracy: 0.8530
Epoch 2/100
250/250 [==============================] - 2s 8ms/step - loss: 0.3574 - accuracy: 0.8553 - val_loss: 0.3533 - val_accuracy: 0.8555
Epoch 3/100
250/250 [==============================] - 2s 8ms/step - loss: 0.3469 - accuracy: 0.8633 - val_loss: 0.3464 - val_accuracy: 0.8615
Epoch 4/100
250/250 [==============================] - 2s 8ms/step - loss: 0.3395 - accuracy: 0.8619 - val_loss: 0.3419 - val_accuracy: 0.8510
Epoch 5/100
250/250 [==============================] - 2s 8ms/step - loss: 0.3348 - accuracy: 0.8622 - val_loss: 0.3440 - val_accuracy: 0.8550
Epoch 6/100
250/250 [==============================] - 2s 8ms/step - loss: 0.3309 - accuracy: 0.8634 - val_loss: 0.3429 - val_accuracy: 0.8510
Epoch 7/100
250/250 [==============================] - 2s 8ms/step - loss: 0.3301 - accuracy: 0.8639 - val_loss: 0.3488 - val_accuracy: 0.851

In [37]:
model.save('model.h5')

c:\Users\ROOT\Desktop\ann classifier\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [42]:
## Load Tensorboard Extention

%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [67]:
%tensorboard --logdir logs/fit/20241122-181050

Reusing TensorBoard on port 6006 (pid 1752), started 0:00:05 ago. (Use '!kill 1752' to kill it.)

In [ ]:
## Load the pickle file
